In [1]:
import os
import ssl
import httpx
from openai import OpenAI

# Configurar cliente com verificação SSL desabilitada
def criar_cliente_openai_ssl_desabilitado():
    # Criar um contexto SSL que não verifica certificados
    ssl_context = ssl.create_default_context()
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE
    
    # Criar cliente httpx com SSL desabilitado
    http_client = httpx.Client(verify=False)
    
    # Configurar cliente Groq
    client = OpenAI(
        api_key=os.getenv("OPENAI_API_KEY"),  # Certifique-se de ter sua chave API configurada
        http_client=http_client
    )
    
    return client

# Testar conexão
try:
    client = criar_cliente_openai_ssl_desabilitado()
    print("Cliente criado com sucesso!")
except Exception as e:
    print(f"Erro ao criar cliente: {e}")

Cliente criado com sucesso!


### Fluxo de Conversa
- User: Usuário que inia a conversa
- Assistant: Bot que esponde a pergunta

In [2]:
mensagens = [{"role": "user", "content": "Defina uma laranja em 5 palavras"}]

response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=mensagens,
    max_tokens=1000,
    temperature=0
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
response

ChatCompletion(id='chatcmpl-CsEGJdHcH3TR6XF4oJl8mLZVtXYTz', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Fruta cítrica suculenta e saborosa.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1767040571, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=14, prompt_tokens=18, total_tokens=32, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [8]:
response.choices[0].message.content

'Fruta cítrica suculenta e saborosa.'

In [9]:
mensagens.append({"role": "assistant", "content": response.choices[0].message.content})

In [10]:
mensagens.append({"role": "user", "content": "E qual é sua cor?"})

In [11]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=mensagens,
    max_tokens=1000,
    temperature=0
)

In [12]:
response.choices[0].message.content

'Laranja.'

In [13]:
mensagens.append({"role": "assistant", "content": response.choices[0].message.content})

In [14]:
mensagens

[{'role': 'user', 'content': 'Defina uma laranja em 5 palavras'},
 {'role': 'assistant', 'content': 'Fruta cítrica suculenta e saborosa.'},
 {'role': 'user', 'content': 'E qual é sua cor?'},
 {'role': 'assistant', 'content': 'Laranja.'}]

### Utilizando Função

In [15]:
def geracao_texto(mensagens, model="gpt-3.5-turbo-0125", max_tokens=1000, temperature=0):
    resposta = client.chat.completions.create(
        messages=mensagens,
        model=model,
        max_tokens=max_tokens,
        temperature=temperature
    )
    mensagens.append({"role": "assistant", "content": resposta.choices[0].message.content})
    return mensagens


In [16]:
msg1 = [{"role": "user", "content": "Defina uma API em 5 palavras"}]
mensagens = geracao_texto(msg1)

In [17]:
mensagens

[{'role': 'user', 'content': 'Defina uma API em 5 palavras'},
 {'role': 'assistant',
  'content': 'Interface para comunicação entre sistemas.'}]

### Utilizando Stream

In [20]:
mensagens = [{"role": "user", "content": "Conte uma história simples e objetiva sobre LLM"}]

resposta = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=mensagens,
    max_tokens=1000,
    temperature=0,
    stream=True
)

In [21]:
resposta

In [22]:
for resposta_stream in resposta:
    texto = resposta_stream.choices[0].delta.content
    if texto:
        print(texto, end="")

LLM era um jovem estudante de direito muito dedicado e determinado. Desde pequeno, ele sonhava em se tornar um advogado renomado e ajudar as pessoas a obter justiça.

Durante seus estudos, LLM se destacava pela sua inteligência e capacidade de argumentação. Ele sempre se esforçava ao máximo para aprender e se aprimorar, buscando sempre novos desafios e oportunidades de crescimento.

Após se formar, LLM conseguiu um emprego em um escritório de advocacia de prestígio, onde rapidamente se destacou pelo seu trabalho árduo e dedicação aos clientes. Sua reputação como advogado competente e ético logo se espalhou, e ele passou a ser procurado por pessoas de todo o país em busca de ajuda jurídica.

Com o passar dos anos, LLM se tornou um dos advogados mais respeitados do país, conhecido por sua integridade, competência e comprometimento com a justiça. Ele continuou a ajudar as pessoas a obterem seus direitos e a lutar por um mundo mais justo e igualitário.

LLM nunca se esqueceu de suas origen